In [1]:
#4.1.1 Dynamic Threshold

In [2]:
import numpy as np
import matplotlib.pyplot as mpl
import scipy.ndimage as ndimage
import skimage.filter as skif

/Users/AlexCen1/anaconda/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [3]:
#Generating data points with a non-uniform background
x = np.random.uniform(low=0,high=100,size=20).astype(int)
y = np.random.uniform(low=0,high=100,size=20).astype(int)

In [4]:
#Creating image with non-uniform background
func = lambda x,y:x**2+y**2
grid_x,grid_y = np.mgrid[-1:1:100j,-2:2:100j]
bkg = func(grid_x,grid_y)
bkg = bkg/np.max(bkg)

In [6]:
#Creating points
clean = np.zeros((100,100))
clean[(x,y)] += 5
clean = ndimage.gaussian_filter(clean,3)
clean = clean/np.max(clean)

In [7]:
#Combining both the non-uniform background and points
fimg = bkg+clean
fimg = fimg/np.max(fimg)

In [9]:
#Defining minimum neighboring size of objects
block_size = 3

In [10]:
#Adaptive threshold function which returns image
#map of structures that are different relative to background
adaptive_cut = skif.threshold_adaptive(fimg,block_size,offset=0)

In [16]:
#Global threshold 
global_thresh = skif.threshold_otsu(fimg)
global_cut = fimg>global_thresh

In [12]:
#Creating figure to highlight difference between 
#adaptive and global threshold methods
fig = mpl.figure(figsize=(8,4))
fig.subplots_adjust(hspace=0.05,wspace=0.05)

In [13]:
ax1 = fig.add_subplot(131)
ax1.imshow(fimg)
ax1.xaxis.set_visible(False)
ax1.yaxis.set_visible(False)

In [17]:
ax2 = fig.add_subplot(132)
ax2.imshow(global_cut)
ax2.xaxis.set_visible(False)
ax2.yaxis.set_visible(False)

In [18]:
ax3 = fig.add_subplot(133)
ax3.imshow(adaptive_cut)
ax3.xaxis.set_visible(False)
ax3.yaxis.set_visible(False)

In [19]:
fig.savefig('scikit_image_f01.pdf',bbox_inches='tight')

In [ ]:
##4.1.2 Local Maxima

In [20]:
import numpy as np
import matplotlib.pyplot as mpl
import scipy.ndimage as ndimage
import skimage.morphology as morph

In [21]:
#Generating data points with a non-uniform background
x = np.random.uniform(low=0,high=200,size=20).astype(int)
y = np.random.uniform(low=0,high=200,size=20).astype(int)

In [22]:
#Creating image with non-uniform background
func = lambda x,y:np.cos(x)+np.sin(y)
grid_x,grid_y = np.mgrid[0:12:200j,0:24:400j]
bkg = func(grid_x,grid_y)
bkg = bkg/np.max(bkg)

In [23]:
#Creating points
clean = np.zeros((200,400))
clean[(x,y)] += 5
clean = ndimage.gaussian_filter(clean,3)
clean = clean/np.max(clean)

In [24]:
#Combining both the non-uniform background and points
fimg = bkg+clean
fimg = fimg/np.max(fimg)

In [25]:
#Calculating local maxima
lm1 = morph.is_local_maximum(fimg)
x1,y1 = np.where(lm1.T==True)

AttributeError: 'module' object has no attribute 'is_local_maximum'

In [26]:
#Creating figure to show local maximum detection rate success
fig = mpl.figure(figsize=(8,4))

In [27]:
ax = fig.add_subplot(111)
ax.imshow(fimg)
ax.scatter(x1,y1,s=100,facecolor='none',edgecolor='#009999')
ax.set_xlim(0,400)
ax.set_ylim(0,200)
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
fig.savefig('scikit_image_f02.pdf',bbox_inches='tight')

NameError: name 'x1' is not defined

In [1]:
import numpy as np
import pyfits
import matplotlib.pyplot as mpl
import skimage.morphology as morph
import skimage.exposure as skie

ImportError: No module named pyfits

# 4.2 Scikit-Learn

In [2]:
import numpy as np
import matplotlib.pyplot as mpl
from mpl_toolkits.mplot3d import Axes3D
from sklearn import linear_model
from sklearn.datasets.samples_generator import make_regression

In [3]:
#Generating synthetic data for training and testing 
X,y = make_regression(n_samples=100,n_features=2,n_informative=1,\
                     random_state=0,noise=50)

In [4]:
#X and y are values for 3D space.We first need to train 
#the machine, so we split X and y into X_train, X_test,
#y_train, and y_test. The *_train data will be given to the model to train it.
X_train, X_test = X[:80],X[-20:]
y_train, y_test = y[:80],y[-20:]

In [5]:
#Creating instance of model
regr = linear_model.LinearRegression()

In [6]:
#Training the model
regr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [8]:
#Printing the coefficients
regr.coef_

array([-10.25691752,  90.5463984 ])

In [9]:
#Predicting y-value based on training
X1 = np.array([1.2,4])
regr.predict(X1)

array([ 350.86036386])

In [10]:
#With the *_test data we can see how the result matches 
#the data the model was trained with.
#It should be a good match as the *_train and *_test
#data come from the same sample. Output: 1 is perfect
#prediction and anything lower is worse.
regr.score(X_test,y_test)

0.94982749226079577

In [11]:
fig = mpl.figure(figsize=(8,5))
ax = fig.add_subplot(111,projection='3d')
#ax = Axes3D(fig)

In [12]:
#Data
ax.scatter(X_train[:,0],X_train[:,1],y_train,facecolor='#00CC00')
ax.scatter(X_test[:,0],X_test[:,1],y_test,facecolor='#FF7800')

In [13]:
#Function with coefficient variables
coef = regr.coef_
line = lambda x1,x2:coef[0]*x1+coef[1]*x2
grid_x1,grid_x2 = np.mgrid[-2:2:10j,-2:2:10j]
ax.plot_surface(grid_x1,grid_x2,line(grid_x1,grid_x2),alpha=0.1,color='k')
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.zaxis.set_visible(False)
fig.savefig('scikit_learn_regression.pdf',bbox='tight')

In [ ]:
##4.2.2 Clustering

In [16]:
import numpy as np
import matplotlib.pyplot as mpl
from scipy.spatial import distance
from sklearn.cluster import DBSCAN

In [18]:
#Creating data
c1 = np.random.randn(100,2)+5
c2 = np.random.randn(50,2)

In [19]:
#Creating a nuiformly distributed background
u1 = np.random.uniform(low=-10,high=10,size=100)
u2 = np.random.uniform(low=-10,high=10,size=100)
c3 = np.column_stack([u1,u2])

In [20]:
#pooling all the data into one 150x2 array
data = np.vstack([c1,c2,c3])

In [23]:
#Calculating the cluster with DBSCAN function.
#db.labels_ is an array with identifiers to the 
#different clusters in the data.
db = DBSCAN().fit(data)
labels = db.labels_

In [25]:
#Retrieving coordinates for points in each
#identified core. There are two clusters
#denoted as 0 and 1 and the noise is denoted
#as -1. Here we split the data based on which 
#component they belong to.
dbc1 = data[labels==0]
dbc2 = data[labels==1]
noise = data[labels==-1]

In [26]:
#Setting up plot details
x1,x2 = -12,12
y1,y2 = -12,12

In [27]:
fig = mpl.figure()
fig.subplots_adjust(hspace=0.1,wspace=0.1)

In [33]:
ax1 = fig.add_subplot(121,aspect='equal')
ax1.scatter(c1[:,],c1[:,1],lw=0.5,color='#00CC00')
ax1.scatter(c2[:,],c2[:,1],lw=0.5,color='#o28E9B')
ax1.scatter(c3[:,],c3[:,1],lw=0.5,color='#FF78oo')
ax1.xaxis.set_visible(False)
ax1.yaxis.set_visible(False)
ax1.set_xlim(x1,x2)
ax1.set_ylim(y1,y2)
ax1.text(-11,10,'Original')

ValueError: x and y must be the same size

In [34]:
ax2 = fig.add_subplot(122,aspect='equal')
ax2.scatter(dbc1[:,0],dbc1[:,1],lw=0.5,color='#ooCCoo')
ax2.scatter(dbc2[:,0],dbc2[:,1],lw=0.5,color='#o28E9B')
ax2.scatter(noise[:,0],noise[:,1],lw=0.5,color='#FF78oo')
ax2.xaxis.set_visible(False)
ax2.yaxis.set_visible(False)
ax2.set_xlim(x1,x2)
ax2.set_ylim(y1,y2)
ax2.text(-11,10,'DBSCAN identified')
fig.savefig('scikit_learn_cluster.pdf',bbox_inches='tight')

ValueError: to_rgba: Invalid rgba arg "#"
to_rgb: Invalid rgb arg "#"
invalid hex color string "#"